In [1]:
# from keras import backend as K
# K.set_image_dim_ordering('tf')
import os
# import tensorflow as tf
import numpy as np
import scipy.io
import time
import datetime
import pandas as pd
from xgboost import XGBClassifier

# from sklearn.model_selection import train_test_split, StratifiedKFold

# from scipy.interpolate import griddata
# from sklearn.preprocessing import scale
# from functools import reduce
# from keras.layers import Conv3D, MaxPool3D, Flatten, Dense, Conv2D, MaxPooling2D, Conv1D, MaxPool1D
# from keras.models import Sequential
# from keras.layers import Dropout, Input, BatchNormalization
# from sklearn.metrics import confusion_matrix, accuracy_score
# # from plotly.offline import iplot, init_notebook_mode
# from keras.losses import categorical_crossentropy
# from keras.optimizers import Adadelta
# # import plotly.graph_objs as go
# # from matplotlib.pyplot import cm
# # from keras.models import Model
# import numpy as np

# import keras
# # import h5py
# from keras.utils import to_categorical
# from sklearn.model_selection import cross_val_score
# from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
# def make_matrix(df):
# #     mat=np.array(df[1,:])
#     return df.values

In [3]:
# def reformatInput(data, labels):
#     indices = np.random.permutation(2131)

#     trainIndices = [indices[:int(2131*.8)]]
#     validIndices = [indices[int(2131*.8):]]

#     if data.ndim == 3:
#         return [(data[trainIndices], np.squeeze(labels[trainIndices]).astype(np.int32)),
#                 (data[validIndices], np.squeeze(labels[validIndices]).astype(np.int32))]
# #                 (data[testIndices], np.squeeze(labels[testIndices]).astype(np.int32))]
#     elif data.ndim == 5:
#         return [(data[:, trainIndices], np.squeeze(labels[trainIndices]).astype(np.int32)),
#                 (data[:, validIndices], np.squeeze(labels[validIndices]).astype(np.int32))]

In [4]:
df=pd.read_csv('emotions.csv')

In [5]:
df.sample(5)

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
1887,-0.753,5.74,16.800,-10.90,22.6,-0.427,2.320,1.71,-1.620,-7.060,...,-3.96,42.70,42.70,-3.96,52.3,20.50,-25.8,-25.8,20.50,POSITIVE
1379,5.940,24.60,-360.000,5.37,27.5,0.620,0.154,1.25,2.040,-2.930,...,211.00,-83.00,-83.00,211.00,-182.0,160.00,-95.8,-95.8,160.00,NEGATIVE
351,-21.900,-19.40,-0.952,-39.50,-56.2,-40.200,-0.511,-94.70,-42.500,9.240,...,-41.70,126.00,126.00,-41.70,-113.0,1.42,23.4,23.4,1.42,POSITIVE
244,31.100,31.70,32.900,29.40,25.0,-1.960,2.090,-4.32,-2.230,-0.312,...,-6.87,8.37,8.37,-6.87,222.0,5.48,-65.1,-65.1,5.48,NEUTRAL
1481,10.500,31.40,-196.000,21.10,26.5,12.400,-0.306,57.00,0.458,-2.790,...,467.00,-146.00,-146.00,467.00,82.8,76.10,-27.4,-27.4,76.10,NEGATIVE


In [6]:
df['label'].replace(['POSITIVE', 'NEUTRAL','NEGATIVE'],[2,1,0], inplace= True )    

In [7]:
df.sample(5)

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
184,-4.43,6.07,22.1,-12.90,23.3,0.0727,-2.220,3.110,1.05,-2.88,...,12.800,-12.800,-12.800,12.800,106.0,7.54,-19.3,-19.3,7.54,2
1567,13.30,24.70,-128.0,12.70,27.7,0.1560,1.860,-3.700,-3.58,-6.59,...,-166.000,50.900,50.900,-166.000,-52.9,26.70,10.9,10.9,26.70,0
1935,14.00,22.90,-268.0,8.38,26.2,1.8000,0.699,-2.760,2.31,3.86,...,237.000,-65.800,-65.800,237.000,83.2,28.20,-34.2,-34.2,28.20,0
459,29.30,28.80,30.3,29.20,25.7,1.2900,0.638,0.396,1.97,-1.34,...,12.500,-2.600,-2.600,12.500,212.0,35.10,-69.7,-69.7,35.10,1
1814,26.20,31.20,29.4,24.90,26.0,-7.8700,-0.724,9.500,-6.24,-6.88,...,-0.291,0.134,0.134,-0.291,121.0,11.20,31.7,31.7,11.20,1


In [8]:
dftrain=df.iloc[:,:-1]
dftest=df.iloc[:,-1]
dftrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2132 entries, 0 to 2131
Columns: 2548 entries, # mean_0_a to fft_749_b
dtypes: float64(2548)
memory usage: 41.4 MB


In [9]:
# dftrain=make_matrix(dftrain)
# dftest=make_matrix(dftest)

In [10]:
# dftrain=np.asarray(df)
# dftest=np.asarray(dftest)



In [11]:
# print(dftrain.shape)
# print('*'*100)
# print(dftest.shape)

In [12]:
# X_train,X_test,y_train,y_test= train_test_split(dftrain,dftest,test_size=.20, random_state=0)
# X_train.shape
# X_test.shape
# # y_train.shape[0]


In [13]:
# n_cols = X_train.shape[1]
# model_mc = Sequential()

# #add model layers
# model_mc.add(Dense(500, activation='relu', input_shape=(n_cols,)))
# model_mc.add(Dense(400, activation='relu'))
# model_mc.add(Dense(300, activation='relu'))
# model_mc.add(Dense(1))

# #compile model using mse as a measure of model performance
# model_mc.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
# #train model
# model_mc.fit(X_train, y_train, validation_split=0.2, epochs=30, )

In [14]:
# X_train=X_train.reshape(1705,2549)
# print(X_train.shape)
# X_test=X_test.reshape(427,2549)
# print(X_test.shape)
# fit and evaluate a model
# def evaluate_model(trainX, trainy, testX, testy):
# 	verbose, epochs, batch_size = 0, 10, 32
# 	n_timesteps, n_features, n_outputs = trainX.shape[0], trainX.shape[1], trainy.shape[0]
# 	model = Sequential()
# 	model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(2549,1)))
# # 	model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# # 	model.add(Dropout(0.5))
# # 	model.add(MaxPool1D(pool_size=2))
# 	model.add(Flatten())
# 	model.add(Dense(100, activation='relu'))
# 	model.add(Dense(3, activation='softmax'))
# 	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# 	# fit network
# 	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
# 	# evaluate model
# 	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
# 	return accuracy
# ans= evaluate_model(X_train, y_train, X_test, y_test)
# # fit and evaluate a model

In [15]:
# from keras import backend as K
# K.set_image_dim_ordering('tf')
# def make_model():
  
#   num_category = 2
#   # t_train=y_train
#   # y_test=y_val
#   # y_train = keras.utils.to_categorical(y_train, num_category)
#   # y_test = keras.utils.to_categorical(y_val, num_category)
#   model = Sequential()
#   #convolutional layer with rectified linear unit activation
#   model.add(Conv1D(32, kernel_size=3,
#                    activation='relu'))
#   #32 convolution filters used each of size 3x3
#   #again
#   model.add(MaxPooling1D(pool_size=2))
#   model.add(BatchNormalization())
#   model.add(Dropout(0.1))
#   model.add(Conv1D(64, 3, activation='relu'))
#   #64 convolution filters used each of size 3x3
#   #choose the best features via pooling
#   model.add(MaxPooling1D(pool_size=2))
#   model.add(BatchNormalization())
#   model.add(Dropout(0.2))
#   model.add(Conv1D(128, kernel_size=3,
#   #                  activation='relu'))
#   model.add(MaxPooling1D(pool_size=2))
# #   model.add(BatchNormalization())
#   model.add(Dropout(0.2))
#   # randomly turn neurons on and off to improve convergence
#   # model.add(Dropout(0.25))
#   # model.add(Conv2D(256, (3, 3), activation='relu'))
#   # #64 convolution filters used each of size 3x3
#   # #choose the best features via pooling
#   # model.add(MaxPooling2D(pool_size=(2, 2)))
#   # model.add(BatchNormalization())
#   # model.add(Dropout(0.2))
#   # model.add(Conv2D(512, kernel_size=(3, 3),
#   #                  activation='relu'))
#   # model.add(MaxPooling2D(pool_size=(2, 2)))
#   # model.add(BatchNormalization())
#   # model.add(Dropout(0.1))
#   # # randomly turn neurons on and off to improve convergence
#   # model.add(Dropout(0.25))
#   # # flatten since too many dimensions, we only want a classification output
#   model.add(Flatten())
#   #fully connected to get all relevant data
#   model.add(Dense(128, activation='relu'))
#   #one more dropout for convergence' sake :) 
#   model.add(Dropout(0.2))
#   #output a softmax to squash the matrix into output probabilities
#   model.add(Dense(num_category, activation='softmax'))
#   print(model.summary())
#   model.compile(loss=keras.losses.binary_crossentropy,
#               optimizer=keras.optimizers.Adam(.0001),
#               metrics=['accuracy'])
#   return model
# model=make_model()

In [16]:
#machine learning
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split #to create validation data set

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

X_training, X_valid, y_training, y_valid = train_test_split(dftrain, dftest, test_size=0.20, random_state=0)

# from sklearn.decomposition import PCA
# pca = PCA(n_components = 3)
# X_training = pca.fit_transform(X_training)
# X_valid = pca.transform(X_valid)
# explained_variance = pca.explained_variance_ratio_
# print (explained_variance)

# from sklearn import svm
# from sklearn.model_selection import GridSearchCV
# # X,y, nfolds= X_train, y_train,10
# def svc_param_selection(X, y, nfolds):
#   Cs = [2**-10,2**-8,2**-6,2**-4,2**-2,2**0,2**2,2**4,2**6,2**8,2**10]
#   gammas = [2**-10,2**-8,2**-6,2**-4,2**-2,2**0,2**2,2**4,2**6,2**8,2**10]
#   param_grid = {'C': Cs, 'gamma' : gammas}
#   grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
#   grid_search.fit(X, y)
#   grid_search.best_params_
#   return grid_search
# s= svc_param_selection(X_training, y_training, nfolds=10)

# s.fit(X_training,y_training)
# pred_rf = s.predict(X_valid)
# acc_rf = accuracy_score(y_valid, pred_rf)
# print(acc_rf)

In [17]:
# rf_clf = RandomForestClassifier()
rf_clf = XGBClassifier()

rf_clf.fit(X_training, y_training)
pred_rf = rf_clf.predict(X_valid)
acc_rf = accuracy_score(y_valid, pred_rf)
print(acc_rf)
# pred_rf

# rf_clf = RandomForestClassifier()

# parameters = {"n_estimators": [4, 5, 6, 7, 8, 9, 10, 15], 
#               "criterion": ["gini", "entropy"],
#               "max_features": ["auto", "sqrt", "log2"], 
#               "max_depth": [2, 3, 5, 10], 
#               "min_samples_split": [10],
#               "min_samples_leaf": [1, 5, 8, 10]
#              }

# grid_cv = GridSearchCV(rf_clf, parameters, scoring = make_scorer(accuracy_score))
# grid_cv = grid_cv.fit(X_training, y_training)

# print("Our optimized Random Forest model is:")
# grid_cv.best_estimator_

# rf_clf = grid_cv.best_estimator_
# rf_clf.fit(X_training, y_training)
# pred_rf = rf_clf.predict(X_valid)
# acc_rf = accuracy_score(y_valid, pred_rf)
# print(acc_rf)

KeyboardInterrupt: 